# Encuesta Nacional de la Dinámica Demógrafica.



##  Diccionario de Datos 

Como primer objetivo, vamos a extraer los diccionarios de datos.

### Descarga
Para la descarga vamos a recuperar el url del archivo. Primero descargaremos las librerías correspondientes y haremos la conexión.

In [489]:

"""
import requests as r
from bs4 import BeautifulSoup

enadid_url = "https://www.inegi.org.mx/programas/enadid/2018/"

tabs = ['documentacion','tabulados','microdatos', 'datos_abiertos','herramientas']

url = enadid_url + "#" + tabs[2]

response = r.get(url)

"""


'\nimport requests as r\nfrom bs4 import BeautifulSoup\n\nenadid_url = "https://www.inegi.org.mx/programas/enadid/2018/"\n\ntabs = [\'documentacion\',\'tabulados\',\'microdatos\', \'datos_abiertos\',\'herramientas\']\n\nurl = enadid_url + "#" + tabs[2]\n\nresponse = r.get(url)\n\n'

Una vez la conexión esta hecha. Entonces partimos a extraer el texto.

In [490]:
import numpy as np
import pandas as pd

enadid = pd.read_excel('fd_enadid_2018_2014_2018.xlsx', sheet_name=None)
dictsheets = enadid.keys()
dictsheets

dict_keys(['TVIVIENDA', 'THOGAR', 'TSDEM', 'TMIGRANTE', 'TFECHISEMB', 'TMUJER1', 'TMUJER2'])

## Analísis
Una vez hayamos logrado descargar el diccionario de datos. Nuestro objetivo es crear un documento que almacene la mayor cantidad de variables posibles. Además vimos que un diccionario posee varias hojas. Entonces partiremos de la primera hoja. 

In [491]:
test_analysis = enadid['TMIGRANTE']
test_analysis.head()

,Unnamed: 0,INEGI. Encuesta Nacional de la Dinámica Demográfica 2014-2018. Descriptor de base de explotación,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,ESTRUCTURA DEL ARCHIVO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Encuesta Nacional de la Dinámica Demográfica 2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Prueba estadística ENADID 2014-2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Tabla de datos: ENADID_2018-2014.TMIGRANTE_PE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Vemos varias cosas que no nos interesan. Por ejemplo, el texto que aparece en los primeros renglones.

In [492]:
def t_header():
    indicate_remove = test_analysis[
    'INEGI. Encuesta Nacional de la Dinámica Demográfica 2014-2018. Descriptor de base de explotación '].isna()
    indicate_remove = list(indicate_remove)
    indicate_remove
    remove = 0
    t = 0
    while remove < 3:
        if indicate_remove[t] == True:
            remove += 1
        t += 1
    return t

def remove_header(t):
    remove_index = np.arange(0, t)
    test_analysis = test_analysis.drop(index = remove_index)

Ahora, eliminamos los renglones vacíos.

In [493]:
test_analysis['Unnamed: 0'] = test_analysis.isna().sum(axis = 1)
n_cols = test_analysis['Unnamed: 0'].max()
# test_analysis = test_analysis[test_analysis['Unnamed: 0'] < n_cols]

Ahora debemos seleccionar las categórias de cada una de las tablas que aparecen.

In [494]:
sub_tables = test_analysis[test_analysis['Unnamed: 0'] == (n_cols - 1)]
sub_tables = sub_tables[~sub_tables['INEGI. Encuesta Nacional de la Dinámica Demográfica 2014-2018. Descriptor de base de explotación '].isna()]
names = sub_tables['INEGI. Encuesta Nacional de la Dinámica Demográfica 2014-2018. Descriptor de base de explotación '].to_list()
sub_index = np.array(sub_tables.index)
sub_index = sub_index[sub_index > t_header()]


In [495]:
subdfs = []
for i in range(len(sub_index) - 1):
    subdfs.append(test_analysis.iloc[sub_index[i]:sub_index[i + 1],:])
    if sub_index[i + 1] == sub_index[-1]:
        subdfs.append(test_analysis.iloc[sub_index[i + 1]:,:])

Ahora, que las tenemos en una lista. Procedemos a limpiarlas.

In [496]:
def df_block(ldf):
    superdf = pd.DataFrame()
    for q, df_test in enumerate(ldf):
        # print(q)
        df_test = subdfs[q]
        df_cols = list(df_test.iloc[1,:])
        if df_cols[2] == 'Nemónico_PE':
            df_cols[2] = 'Mnemónico_PE'
        df_test.columns = df_cols
        df_name = df_test.iloc[0, 1]
        if q == 0:
            df_test = df_test.drop(columns = df_test.columns[0]).iloc[4: ,:]
        else:
            df_test = df_test.drop(columns = df_test.columns[0]).iloc[3: ,:]
        df_clean = df_test.dropna(axis= 0, how='all').ffill()
        df_clean['SubNombre'] = df_name
        # print(df_clean)
        superdf = pd.concat([superdf, df_clean])
    return superdf

# dfls = df_block(subdfs)


In [497]:
dfls = df_block(subdfs)



Finalmete, lo guardamos en un excel llamado . result.xlsx.

In [498]:
dfls.set_index('ID',inplace=True)
dfls.to_excel('result_test8.xlsx')

De las pruebas, vimos que hay problemas cuando cambian las columnas. Entonces lo que haremos será asegurarnos que se mantenga uniforme. Primero definimos la 

In [499]:
def addtext_tolist(lista: list, texto: str,lado: str):
    for i in range(len(lista)):
        if lado == 'right':
            lista[i] = lista[i] + texto
        else:
            lista[i] = texto + lista[i]
    return lista

def comparelists(listaA: list, listaB: list):
    d_index = []
    if len(listaA) == len(listaB):
        for i in range(len(listaA)):
            if listaA[i] != listaB[i]:
                d_index.append(i)
    return d_index

def couple_cols(actual: pd.DataFrame, new: pd.DataFrame):
    now_cols = list(actual.columns)
    new_cols = list(new.columns)
    if len(now_cols) != len(new_cols):
        return -1
    for i in range(len(now_cols)):
        now_cols[i] = str(now_cols[i]) 
        new_cols[i] = str(new_cols[i])
    mod_cols = comparelists(now_cols, new_cols)
    new_cols = {new_cols[i] : now_cols[i] for i in mod_cols}
    new = new.rename(columns=new_cols)
    return new


In [500]:
def df_blocks(ldf):
    superdf = pd.DataFrame()
    for q, df_test in enumerate(ldf):
        # print(q)
        df_cols = list(df_test.iloc[1,:])
        if df_cols[2] == 'Nemónico_PE':
            df_cols[2] = 'Mnemónico_PE'
        df_test.columns = df_cols
        df_name = df_test.iloc[0, 1]
        df_test.loc[:, ['SubNombre']] = df_name
        if q == 0:
            df_test = df_test.drop(columns = df_test.columns[0]).iloc[4: ,:]
        else:
            df_test = df_test.drop(columns = df_test.columns[0]).iloc[3: ,:]
            df_test = couple_cols(superdf, df_test)
        df_clean = df_test.dropna(axis= 0, how='all').ffill()
        # print(df_clean)
        superdf = pd.concat([superdf, df_clean])
    return superdf

dfls = df_blocks(subdfs)



In [501]:
print(dfls.columns)
dfls.set_index('ID',inplace=True)
dfls.to_excel('result_test8.xlsx')

Index([             'ID',    'Mnemónico_PE',          'Nombre',
           'Descripción',            'Tipo',           'Long.',
       'Códigos válidos',          'Clases',               nan,
              'Catálogo',     'Comentarios',       'SubNombre'],
      dtype='object')


Ahora si, modificaremos el código anterior para así poder leer cualquier base.

In [502]:

print(before_df.columns)
print(actual_df.columns)

x = list(before_df.columns)
y = list(actual_df.columns)



mod_col = comparelists(x, y)
new_colnames = {y[i] : x[i] for i in mod_col}
actual_df = actual_df.rename(columns = new_colnames)
print(new_colnames)
actual_df

Index([                2,              'ID',    'Mnemónico_PE',
                'Nombre',     'Descripción',            'Tipo',
                 'Long.', 'Códigos válidos',          'Clases',
                     nan,        'Catálogo',     'Comentarios'],
      dtype='object')
Index([                2,              'ID',    'Mnemónico_PE',
                'Nombre',     'Descripción',            'Tipo',
                 'Long.', 'Códigos válidos',          'Clases',
                     nan,        'Catálogo',     'Comentarios'],
      dtype='object')
{nan: nan}


,2,ID,Mnemónico_PE,Nombre,Descripción,Tipo,Long.,Códigos válidos,Clases,NaN,Catálogo,Comentarios
43,11,Sección IV. Migración internacional,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,2,ID,Mnemónico_PE,Nombre,Pregunta,Tipo,Long.,Códigos válidos,Clases,NaN,Catálogo,Comentarios
45,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Valor,Etiqueta,NaN,NaN
46,3,11,P4_4,Condición de residencia,"Cuando (NOMBRE) se fue la última vez, ¿vivía c...",Caracter,1,"1,2",1,Sí,NaN,NaN
47,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,No,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
160,3,39,P4_20_1,Residencia actual,(NOMBRE) actualmente forma parte de este hogar?,Caracter,1,"1,2",1,Sí,NaN,NaN
161,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,No,NaN,NaN
162,5,40,P4_20_2,Numero de renglón del migrante,Número de renglón en la lista de personas,Caracter,2,01...30,NaN,NaN,NaN,NaN
163,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
